In [1]:
import requests
from bs4 import BeautifulSoup
import Constants
from Constants import classes_amazon
import pprint

In [2]:
book_details = {
    'book': 'shoe dog',
    'author': 'phil knight'
}
pp = pprint.PrettyPrinter(indent=4)

In [3]:
def prepare_url_amazon(dict_book_details):
    return f'https://www.amazon.in/s?k=' + dict_book_details['book'] + ' book'

In [4]:
def get_amazon_html():
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 "
                      "Safari/537.36"
    }
    return requests.get(prepare_url_amazon(dict_book_details=book_details), headers=headers).text

In [5]:
def get_data_from_amazon_html(response):
    soup = BeautifulSoup(response, 'lxml')

    results = soup.find_all('div', {'class': classes_amazon[Constants.ROOT]})
    relevant_matches = []
    for index, result in enumerate(results):
        dictionary = {}

        # fetch product title
        book_title = result.find(class_=classes_amazon[Constants.BOOK_TITLE]).text

        # fetch author
        author_exists = result.find(class_=classes_amazon[Constants.AUTHOR])
        if author_exists:
            author = author_exists.text
            if author.lower() == book_details[Constants.AUTHOR]:
                dictionary[Constants.AUTHOR] = author
                dictionary[Constants.BOOK_TITLE] = book_title

                # fetch rating
                rating_exists = result.find(class_=classes_amazon[Constants.RATING])
                if rating_exists:
                    rating_book_reviews = rating_exists.text
                    dictionary[Constants.RATING] = rating_book_reviews

                # fetch number of reviews
                numReviews_exists = result.find(class_=classes_amazon[Constants.NUM_REVIEWS])
                if numReviews_exists:
                    num_book_reviews = numReviews_exists.text
                    dictionary[Constants.NUM_REVIEWS] = num_book_reviews

                # fetch prices
                discPrice_exists = result.find(class_=classes_amazon[Constants.PRICE][0])
                if discPrice_exists:
                    dictionary[Constants.DISC_PRICE] = discPrice_exists.find(
                        class_=classes_amazon[Constants.PRICE][1]).text[1:]

                    if len(discPrice_exists.find_all(class_=classes_amazon[Constants.PRICE])) > 1:
                        dictionary[Constants.OG_PRICE] = \
                        discPrice_exists.find_all(class_=classes_amazon[Constants.PRICE])[1].text[1:]

                # fetch prime availability
                prime_exists = result.find(class_=classes_amazon[Constants.PRIME])
                if prime_exists:
                    dictionary[Constants.PRIME] = 'yes'
                relevant_matches.append(dictionary)
    return relevant_matches

In [6]:
data_amazon = get_data_from_amazon_html(get_amazon_html())

In [7]:
pp.pprint(data_amazon)

[   {   'author': 'Phil Knight',
        'bookTitle': 'Shoe Dog: A Memoir by the Creator of Nike',
        'discPrice': '1,849',
        'numReviews': '21,445',
        'ogPrice': '1,999',
        'rating': '4.6 out of 5 stars'},
    {   'author': 'Phil Knight',
        'bookTitle': 'DRM Shoe Dog: A Memoir by Phil Knight (Bookmarks '
                     'iincluded)',
        'discPrice': '299',
        'ogPrice': '499'},
    {   'author': 'Phil Knight',
        'bookTitle': 'Shoe Dog (Hindi Edition)',
        'discPrice': '216.60',
        'numReviews': '106',
        'rating': '4.5 out of 5 stars'},
    {   'author': 'Phil Knight',
        'bookTitle': 'Shoe Dog: Young Readers Edition',
        'discPrice': '384',
        'numReviews': '455',
        'ogPrice': '599',
        'rating': '4.7 out of 5 stars'},
    {   'author': 'Phil Knight',
        'bookTitle': 'Shoe Dog: Die offizielle Biografie des NIKE-Gründers '
                     '(German Edition)',
        'discPrice': '739.5